In [1]:
import numpy as np
import matplotlib.pyplot as plt

mu = 0.001
sigma = 0.01
start_price = 5
sr=5

def StockpriceGBM(T,S0,mu,sigma,d=1):
  WienerDifference = np.random.normal(0,T,d)
  return S0*np.exp((mu - 0.5*sigma**2)*T + sigma*WienerDifference)